# DESCRIPCIÓN SISTEMA LÁSER REMOTO





- Canal emisor: compuesto por un láser de estado sólido y un emisor
- Canal receptor: telescopio de apertura a determinar

Vamos a simular primero la cadena de bombeo de Laser Raman, es decir, la ruta que va desde el laser de emisión hasta la muestra y que incluye los conectores, los switches, la fibra óptica y la lente de focalización.

### Datos del láser


In [ ]:
% Energia por pulso
E_pulso = 20;       % mJ 
% Ancho de pulso
tau_pulso = 10;    % ns
% Frecuencia de repetición del láser
frp = 20;               
% Número de pulsos emitidos
num_pulsos = 20; 

theta_L = 3;          % mrad
lambda_L = 532;  % nm
% Diametro del haz de salida, típico de una varilla Nd YAG doblada
diam_haz = 3;      % mm

Pot_Promedio = E_pulso * frp;
% Eficiencia de conversión de las especificaciones de Monocrom, caso genérico
eff_conversion = 0.05;



In [ ]:
% Potencia eléctrica
Pot_electrica = Pot_Promedio / eff_conversion;    %mW
Pot_electrica = Pot_electrica * 1e-3;    %W

In [ ]:
% Tiempo que dura la emisión del láser para tener el tren de pulsos requerido
tau_emitiendo = num_pulsos / frp;

### Datos de propagación

In [ ]:
% Distancia a la que está el blanco
alcance = 1;    % km
% Atenuación atmosférica, día despejado. Depende de las condiciones atmosféricas (pagina 316 de CCD Arrays, Cameras and Displays de Holst)
sigma_atten = 0.4;     % km^-1
% Diametro del haz de la muestra
diam_muestra = 10;     % mm
% Aumentos del expansor
beta_expansor = diam_muestra/diam_haz;
% Potencia pico
Pot_pico = E_pulso * 1e-3 / (tau_pulso *1e-9);     % W emitidos del haz colimado
% Potencia muestra 
% El haz es colimado, no vamos a considerar la atenuación por la distancia al cuadrado
Pot_muestra = Pot_pico * exp(-sigma_atten * alcance);    % W

mancha_real = diam_haz + 2*tan(theta_L*1e-3 / beta_expansor)*alcance*1e5;     % cm






### Calculo de la irradiancia en la muestra

In [ ]:
Irr_muestra_teo = Pot_muestra / (pi * (diam_muestra*1e-1/2)^2);    % W/cm^2
Irr_muestra = Pot_muestra / (pi * (mancha_real*1e-1/2)^2);      % W/cm^2

### Eficiencia raman para la calcita, se emplea como referencia

In [ ]:
lambda = 1086;    % cm^-1
eps_raman_Ca = 1e-8;
lambda_raman_ca = 564.62; % nm
% Irradiancia Raman
Irr_Raman = Irr_muestra*eps_raman_Ca;    % W/cm^2

La muestra se convierte en un emisor lambertiano que emite en π y su flujo debe llegar al cabezal colector que está a una distancia d. 
Convertimos la irradiancia en Radiancia (W/cm^2 str).

In [ ]:
Rad_Raman = Irr_Raman/pi;   % W/cm^2 str

Esta superficie emisora es a todos los efectos puntual para el sistema de colección por lo que debemos calcular la intensidad luminosa.

In [ ]:
I_emitida = Rad_Raman * (pi*(mancha_real*1e-1/2)^2);
% Cálculo de la intensidad óptica       I = L * S

Si supongo que se observa la fuente puntual en eje entonces: I=I0cos(i) ( ver optica instrumental pagina 83).
La Irradiancia en la apertura de la óptica queda como:

In [ ]:
Irr_apertura_optica = I_emitida/(alcance*1e5)^2 * exp(-sigma_atten*alcance);         % W/cm^2

La señal es recogida por el detector que esta a una distancia d.

In [ ]:
% Diametro de apertura óptica
D_opt = 10; % cm
% Transmitancia de la óptica
tau_optica = 0.6;

Flujo_coleccion = tau_optica * Irr_apertura_optica * pi * (D_opt/2)^2;      % W que llegan a la óptica


Nota: ahora suponemos que el sisetma de colección es límite por difracción y que la mancha de airy es más
pequeña que el píxel

### Datos del SPECTROMETRO (de la red de difracción)



In [ ]:
dispersion = 13;     % nm/mm dispersion del espectrometro
eficiencia = 0.5;     % spectrometer throughput

% spectral_resolution_required = 6 , 8 cm^-1   ;          0.17 , 0.23 nm
% spectral_dispersion_required = 12.6 , 17 nm/mm


### Datos del DETECTOR

In [ ]:
OCG = 4.5e-6;       % V/e-
% Tamaño de pixel
px = 0.015;            % mm 
% Números de pixels
N_pixel_H = 2048;
N_pixel_V = 2048;
% Resolución
disp_pix = px * dispersion;


El espectro tiene una anchura de 0.4, 0.5 nm, el cual es recogido en los siguientes pixeles.

In [ ]:
bandwidth_Ca = 0.4;     % nm
% ext = floor*(bandwidth_Ca/disp_pix);      % pixels
ext = (bandwidth_Ca/disp_pix);      % pixels

### Detector data

In [ ]:
% Velocidad de la luz
c = 3*1e8;         % m/s
h = 6.62*1e-34; % J*s
% Eficiencia Cuántica
QE = 0.6;
% Full well capacity
F_wc = 100*1e3;    % e-


Flujo radiante considerado a cada pixel de la señal Raman

In [ ]:
Flujo_px = Flujo_coleccion * eficiencia / round(ext);

Energía Raman recogida por el detector:

In [ ]:
Flujo_px_esp = Flujo_px / bandwidth_Ca;       % W/nm per pixel

Calculamos la energía en el detector empleando $\lambda_{Raman _ Ca}$ = 564.62 nm.
No hace falta hacerlo espectralmente (lo dejamos para saber como hacerlo en otros casos), hemos estado considerándolo siempre de forma discreta:

In [ ]:
% Número de fotoelectrones
Cts_r = Cts_Raman ( lambda_raman_ca*1e-9, Flujo_px_esp, QE);
% disp(Cts_r)

%Cts_r(λ)
function Cts_r = Cts_Raman ( lambda, Flujo_px_esp, QE)
    c = 3*1e8;      h=6.62e-34;
    Cts_r =  Flujo_px_esp / (c * h) *lambda * QE;
end



Los fotoelectrones de la señal Raman son calculados en el rango espectral de la señal:

In [ ]:
fun = @(lambda) Cts_Raman(lambda, Flujo_px_esp*1e9, QE);
lambda_min = (lambda_raman_ca-bandwidth_Ca/2)*1e-9;
lambda_max = (lambda_raman_ca+bandwidth_Ca/2)*1e-9;
Cts_T = integral(fun, lambda_min, lambda_max);

% disp(Cts_T)


El número de fotoelectrones depende del tiempo de integración.
#### La energía por pulso, en cambio, no depende del tiempo.

In [ ]:
% Signal(t_i) = Cts_T     para todo t_i
% Fotoelectrones por segundo
global Signal
Signal = Cts_T;


Transformamos los fotoelectrones detectados a Voltios.

In [ ]:
x = X(1);

function x = X(t, OCG)
    global Signal
    x = Signal*OCG;       %V
end



## Ruido estimado
### a) Dark Current noise


In [ ]:
% signal at T = 20ºC == 293 K
global s_dark
s_dark = 100;               % e-/pixel/s
n_dark = N_dark(293, 1, s_dark); 


function n = N_dark(T, t_i)
    global s_dark
    n = sqrt(s_dark * t_i * (1.4*1e6 * T.^3 .* exp(-9080./T)));
end

### b) Read-out noise

In [ ]:
global N_r
N_r = 7;     % e-

### c) Quantification noise

In [ ]:
global N_q
% bit analogic converter
n = 12;
% Nivel de digitalización para una ganancia normal
DN = F_wc / 2^n;
N_q = DN / sqrt(12);     % e-

### d) Photonic noise 

Señal Raman:

In [ ]:
N_f_Raman = N_f_R(1);     %e-

function n = N_f_R(t_i)
    global Signal
    n = sqrt(Signal);
end


## Ruido total (desviación estándar)

Ruido en función de la temperatura y el tiempo de integración.

In [ ]:
T = linspace(173,330,1);    % K
% Calculamos la variación del ruido total con la temperatura para un intervalo 173 K a 330 K
N_Total = N_T_Total(T,1);     


% Representamos el ruido total
figure(1)
plot(T, N_Total)
title('Total Noise')
xlim([min(T)-1 max(T)+1])
ylim([0 3000])
grid on


function n = N_T_Total (T, t_i)
    global N_q N_r
    n = sqrt(N_q^2 + N_r^2 + N_f_R(t_i)^2+N_dark(T, t_i));
end

Niveles de ruido esperados para la temperatura de operación T = 233 K

In [ ]:
T_op1 = 233;       t_in = [1.5 1 0.5 1.25*1e-3];
N_Total_233K = N_T_Total(T_op1, t_in);


Niveles de ruido esperados para la temperatura de operación T = 263 K

In [ ]:
T_op2 = 263;       
N_Total_263K = N_T_Total(T_op2, t_in);


## Evaluación de SNR para un cierto nivel de señal (Detección Continua)

In [ ]:
T = linspace(193, 293, 101);       
t_i1 = 0.1;     t_i2 = 0.05;    t_i3 = 0.02;    t_i4 = 0.01;
SNR_real_1 = SNR_REAL(T, t_i1);
SNR_real_2 = SNR_REAL(T, t_i2);
SNR_real_3 = SNR_REAL(T, t_i3);
SNR_real_4 = SNR_REAL(T, t_i4);

% SNR
function s = SNR_REAL(T, t_i)
    global Signal 
    s = Signal ./ N_T_Total(T, t_i);
end


In [ ]:
figure(2)
hold on
plot(T, SNR_real_1)
plot(T, SNR_real_2)
plot(T, SNR_real_3)
plot(T, SNR_real_4)
xlabel(' T (K)')
ylabel(' SNR ')
ylim([2397.575 2397.581])
legend('SNR t=0.1', 'SNR t=0.05', 'SNR t=0.02', 'SNR t=0.01', 'Location', 'southwest')
grid on
hold off


## Evaluación de SNR para un cierto nivel (Condición promedio)

Sea N el número de imágenes tomadas (acumulaciones).

In [ ]:
NN = 20;   % Número de pulsos
% t_i = 0.1;
tau_ida_vuelta = 2*alcance*1e3 / c;
SNR_real_A1 = SNR_REAL_A(T, t_i1, NN);
SNR_real_A2 = SNR_REAL_A(T, t_i2, NN);
SNR_real_A3 = SNR_REAL_A(T, t_i3, NN);
SNR_real_A4 = SNR_REAL_A(T, t_i4, NN);


% SNR Average Condition
function s = SNR_REAL_A(T, t_i, N)
    global Signal
    s = Signal * N ./ sqrt(N_T_Total(T, t_i).^2.*N);
end


In [ ]:
% Representamos
figure(3)
hold on
plot(T, SNR_real_A1)
plot(T, SNR_real_A2)
plot(T, SNR_real_A3)
plot(T, SNR_real_A4)
xlabel(' T (K)')
ylabel(' SNR Average Condition ')
ylim([10722.26 10722.32])
legend('SNR_A t=0.1', 'SNR_A t=0.05', 'SNR_A t=0.02', 'SNR_A t=0.01', 'Location', 'southwest')
grid on
hold off
